# Quick intro to Kernel Memory: install, upload a doc, ask a question

This notebook shows the basic usage of Kernel Memory, via the serverless client.

First of all, install the Kernel Memory dependency.

In [ ]:
#r "nuget: Microsoft.KernelMemory.Core, 0.29.240219.2"
#r "nuget: Neo4j.Driver, 5.18.1"
#r "../../src/Neo4jMemoryStorage/bin/Debug/net8.0/Neo4j.KernelMemory.MemoryStorage.dll"

using Microsoft.KernelMemory;
using Neo4j.KernelMemory.MemoryStorage;
using Neo4j.Driver;

For our demo, we use also the "dotenv" nuget to load our 
secret credentials from a `.env` file, expected in the 
parent `examples` directory.


Make sure you create your `.env` file, with this content:

> ```
> OPENAI_API_KEY=<your OpenAI API key>
> NEO4J_URI=neo4j://<your Neo4j URI>
> NEO4J_USERNAME=<your Neo4j user>
> NEO4J_PASSWORD=<your Neo4j password>
> ```

In [ ]:
#r "nuget: dotenv.net, 3.1.3"
using dotenv.net;

var dotEnvOptions = new DotEnvOptions(
        envFilePaths: new[] {"../.env"}
    );

DotEnv.Load(options: dotEnvOptions);
var env = DotEnv.Read(dotEnvOptions);

var neo4jConfig = new Neo4jConfig
{
    Uri = env["NEO4J_URI"],
    Username = env["NEO4J_USERNAME"],
    Password = env["NEO4J_PASSWORD"]
};

Console.WriteLine($"Neo4j URI: {neo4jConfig.Uri}");

Now let's create an instance of Kernel Memory client using the `KernelMemoryBuilder` to configure our solution. 

In this demo we use OpenAI to calculate embeddings and generate text, and the default storage settings, with content and embeddings kept in a volatile memory automatically deleted after the execution.

In [ ]:
var kernelMemory = new KernelMemoryBuilder()
    .WithOpenAIDefaults(env["OPENAI_API_KEY"])
    .WithNeo4j(neo4jConfig)
    .Build<MemoryServerless>();

At this point we have the memory client ready, we can import data and ask questions!

Let's import a PDF file with some details about Kernel Memory:

In [ ]:
await kernelMemory.ImportDocumentAsync("sample-KM-Readme.pdf", documentId: "doc001");

In [ ]:
await kernelMemory.ImportDocumentAsync("../data/deer-horatio.md", documentId: "doc002");

Now, let's ask a question about Kernel Memory:

In [ ]:
var question = "What's Kernel Memory?";

var answer = await kernelMemory.AskAsync(question);

Console.WriteLine($"Question: {question}\n\nAnswer: {answer.Result}");

And here show the sources used to generate the answer above, what we call "citations":

In [ ]:
Console.WriteLine("Sources:\n");

foreach (var x in answer.RelevantSources)
{
    Console.WriteLine($"  - {x.SourceName}  - {x.Link} [{x.Partitions.First().LastUpdate:D}]");
}

In [ ]:
var question = "Who are Deer Horatio and his friend Liv Fawn?";

var answer = await kernelMemory.AskAsync(question);

Console.WriteLine($"Question: {question}\n\nAnswer: {answer.Result}");